In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote # To decode URL
from urllib.parse import urljoin
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import nltk
nltk.download('punkt')       # For tokenization
nltk.download('stopwords')   # For stopwords
nltk.download('wordnet')     # For lemmatizationimport re
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
# Set up stopwords
stop_words = set(stopwords.words('english'))

# # To evade the detection as a bot, pass the argument --disable-blink-features=AutomationControlled as follows
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# variables
youtubeVideo = "https://www.youtube.com/watch?v=jGZzpr4BtAM" 

In [3]:
# URL of the webpage you want to crawl
url = youtubeVideo
# Send an HTTP GET request to the URL
response = requests.get(url)
response

<Response [200]>

In [4]:
# Create a new instance of the Firefox driver
driver = webdriver.Firefox()

In [5]:
# Navigate to the webpage
driver.get(url)

In [6]:
# Youtube
title_attr = {'name': 'title','css_selector' : 'h3[data-qa=\'card-title\']'}
date_attr = {'name': 'date','css_selector': 'span[data-testid=\'timestamp\']'}
link_attri = 'a[data-pb-local-content-field=\'web_headline\']'
class_target = "story-headline"
show_more_button = "#__next > div.wpds-c-iMXAYL > div > main > article > div > div.center.w-100 > div > button"


In [7]:
def get_elements_with_css_selector(url, soup, class_target, decode_urls=True, link_attr = "", others_elements=[]):
    # Use CSS selectors to find all elements with the specified class
    container = soup.find_all(class_ = class_target)

    # Create a list to store the dictionaries
    results = []

    # Loop through the elements and add their text content and link to the list
    for element in container:
        result = {}
        # Use CSS selector to find the link
        if link_attr:
            link_element = element.select_one(link_attr)
            link = link_element['href']
            full_link = urljoin(url, unquote(link)) if decode_urls else link

            result['link'] = full_link

        # If include_attributes is provided, add specified attributes to the result using CSS selectors
        if others_elements:
            for attr in others_elements:
                # Use CSS selector to find the element
                target_element = element.select_one(attr['css_selector'])
                result[attr['name']] = target_element.get_text(strip=True) if target_element else None
        results.append(result)

    return results


In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
container = soup.find_all(class_ = "style-scope ytd-watch-flexy")

In [10]:
# Find the title tag
title_tag = soup.find('meta', property='og:title')

# Return the content of the title tag
if title_tag:
    print(title_tag['content'])

PlayStation 5 vs Xbox Series X: Which is better?


In [14]:
import time
# Scroll down the page by 1000 pixels
SCROLL_PAUSE_TIME = 0.1

# Get scroll height

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollBy(0, 700);")
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

KeyboardInterrupt: 

In [15]:
def get_youtube_comments(driver):
    # Use BeautifulSoup to parse the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Find comments
    comment_divs = soup.select('span.yt-core-attributed-string.yt-core-attributed-string--white-space-pre-wrap')
    comments = [comment.text for comment in comment_divs]
    
    return comments

# Example usage
video_url = url
comments = get_youtube_comments(driver)
print("comments length : ",len(comments))
for comment in comments:
    print(comment)

comments length :  808
PlayStation 5 or Xbox Series X, the next-gen consoles are here. But which is better? Which do I prefer? And which do I think you should buy? This is my honest review and opinion, with one choice.

Which next-gen console do you prefer?
Honestly for most people it comes to whatever your friends have
We all know that the Wii is the most powerful console.
I went with ps5 solely because of dualsense, and I was not disappointed! Though both are amazing and the Xbox controller is more comfortable, i just wanted to experience those games i was in, but you can go with either as they are both truly next gen.
Thanks, SpawnPoiint, for the fantastic comparison video!
Really respect how you just cut to the chase, no overly long intro and just straight talking, not aimed at young children. Subbed, keep up excellent work!
I'm more of an Xbox person, but I still respect PlayStation.
I love your videos and hence a request. Looking at PS Plus and GamePass evolution, could you maybe

In [16]:
df = pandas.DataFrame(comments, columns=['Comments'])
df

,Comments
0,"PlayStation 5 or Xbox Series X, the next-gen c..."
1,Which next-gen console do you prefer?
2,Honestly for most people it comes to whatever ...
3,We all know that the Wii is the most powerful ...
4,"I went with ps5 solely because of dualsense, a..."
...,...
803,"3,095,893"
804,Views
805,2020
806,Nov 24


In [17]:
# Load stopwords
stop_words = set(stopwords.words('english'))

# Initializers for Stemmer and Lemmatizer
lemmatizer = WordNetLemmatizer()
# stemmer = PorterStemmer()

def clean_comment(text):
    if pd.isna(text):
        return text
    # Lowercase normalization
    text = text.lower()
    # Remove URLstk
    # Remove punctuation and non-ASCII
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords and perform /lemmatization
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Rejoin words into one string
    return ' '.join(cleaned_tokens)

# Apply the cleaning function to the comments column
df['Cleaned Comments'] = df['Comments'].apply(clean_comment)

# Display the DataFrame
print(df)


                                              Comments  \
0    PlayStation 5 or Xbox Series X, the next-gen c...   
1                Which next-gen console do you prefer?   
2    Honestly for most people it comes to whatever ...   
3    We all know that the Wii is the most powerful ...   
4    I went with ps5 solely because of dualsense, a...   
..                                                 ...   
803                                          3,095,893   
804                                              Views   
805                                               2020   
806                                             Nov 24   
807  PlayStation 5 or Xbox Series X, the next-gen c...   

                                      Cleaned Comments  
0    playstation xbox series x nextgen console bett...  
1                               nextgen console prefer  
2                 honestly people come whatever friend  
3                            know wii powerful console  
4    went p solely

In [74]:
df_copy = df.copy()
df_copy.drop('Comments', axis=1, inplace=True)
df_copy.to_csv('youcomment.csv', index=False)

In [21]:
# vocabulary
from sklearn.feature_extraction.text import CountVectorizer 
# Create a CountVectorizer object
vectorizer = CountVectorizer()
# Fit and transform the data
X = vectorizer.fit_transform(df['Cleaned Comments'])
# Create a DataFrame
df_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# Display the DataFrame
print(df_words)

     aaa  abes  ability  able  aboust  absolute  absolutely  abysmal  accent  \
0      0     0        0     0       0         0           0        0       0   
1      0     0        0     0       0         0           0        0       0   
2      0     0        0     0       0         0           0        0       0   
3      0     0        0     0       0         0           0        0       0   
4      0     0        0     0       0         0           0        0       0   
..   ...   ...      ...   ...     ...       ...         ...      ...     ...   
803    0     0        0     0       0         0           0        0       0   
804    0     0        0     0       0         0           0        0       0   
805    0     0        0     0       0         0           0        0       0   
806    0     0        0     0       0         0           0        0       0   
807    0     0        0     0       0         0           0        0       0   

     access  ...  youd  youll  young  y

In [22]:
# n-grams
from sklearn.feature_extraction.text import CountVectorizer
# Create a CountVectorizer object
vectorizer = CountVectorizer(ngram_range=(2, 2))
# Fit and transform the data
X = vectorizer.fit_transform(df['Cleaned Comments'])
# Create a DataFrame
df_bigrams = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# Display the DataFrame
print(df_bigrams)

     aaa peace  abes odyssey  ability control  ability play  ability switch  \
0            0             0                0             0               0   
1            0             0                0             0               0   
2            0             0                0             0               0   
3            0             0                0             0               0   
4            0             0                0             0               0   
..         ...           ...              ...           ...             ...   
803          0             0                0             0               0   
804          0             0                0             0               0   
805          0             0                0             0               0   
806          0             0                0             0               0   
807          0             0                0             0               0   

     able find  able get  able install  able play  

In [75]:
from collections import Counter

# Download NLTK stopwords
nltk.download('stopwords')

# Load your dataset
with open("youcomment.csv", "r") as f:
    text = f.read()

# Preprocess text (lowercase, remove punctuation, remove stopwords)
def preprocess_text(text):
    tokens = [word.lower() for word in nltk.word_tokenize(text) if word.isalpha()]
    stopwords = nltk.corpus.stopwords.words('english')
    tokens = [word for word in tokens if word not in stopwords]
    return tokens

processed_text = preprocess_text(text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tariq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [83]:
processed_text

['cleaned',
 'comments',
 'playstation',
 'xbox',
 'series',
 'x',
 'nextgen',
 'console',
 'better',
 'prefer',
 'think',
 'buy',
 'honest',
 'review',
 'opinion',
 'one',
 'choice',
 'nextgen',
 'console',
 'prefer',
 'honestly',
 'people',
 'come',
 'whatever',
 'friend',
 'know',
 'wii',
 'powerful',
 'console',
 'went',
 'p',
 'solely',
 'dualsense',
 'disappointed',
 'though',
 'amazing',
 'xbox',
 'controller',
 'comfortable',
 'wanted',
 'experience',
 'game',
 'go',
 'either',
 'truly',
 'next',
 'gen',
 'thanks',
 'spawnpoiint',
 'fantastic',
 'comparison',
 'video',
 'really',
 'respect',
 'cut',
 'chase',
 'overly',
 'long',
 'intro',
 'straight',
 'talking',
 'aimed',
 'young',
 'child',
 'subbed',
 'keep',
 'excellent',
 'work',
 'im',
 'xbox',
 'person',
 'still',
 'respect',
 'playstation',
 'love',
 'video',
 'hence',
 'request',
 'looking',
 'p',
 'plus',
 'gamepass',
 'evolution',
 'could',
 'maybe',
 'comparison',
 'stack',
 'thank',
 'making',
 'well',
 'done',
 'v

In [89]:
def generate_ngrams(text, n):
  """
  Generates n-grams from a list of words.
  """
  return zip(*[text[i:] for i in range(n)])

# Generate bigrams (2-word sequences)
bigrams = generate_ngrams(processed_text, 2)

# Count bigram frequencies
bigram_counts = Counter(bigrams)


In [98]:
def suggest_word(prefix):
  """
  Suggests words based on the given prefix using bigram counts.
  """
  suggestions = []
  for bigram, count in bigram_counts.items():
    if bigram[0] == prefix:
      print(bigram[0], bigram[1], count)
      suggestions.append(bigram[1])
  return suggestions[:5]  # Limit suggestions to top 5

# Example usage
prefix = "playstation"
suggestions = suggest_word(prefix)
print("Suggestions for", prefix, ":", suggestions)


playstation xbox 12
playstation love 3
playstation gamer 1
playstation online 1
playstation ill 1
playstation im 4
playstation game 5
playstation hard 1
playstation prefer 1
playstation exclusive 13
playstation fanboys 1
playstation one 3
playstation guy 6
playstation pushing 1
playstation p 2
playstation id 2
playstation bought 1
playstation best 3
playstation whats 1
playstation switcher 2
playstation elitist 1
playstation time 2
playstation something 1
playstation never 2
playstation x 1
playstation owned 1
playstation good 1
playstation happy 1
playstation get 1
playstation somehow 1
playstation vr 1
playstation since 5
playstation pc 2
playstation family 1
playstation vastly 1
playstation person 3
playstation owner 1
playstation cuz 1
playstation choose 1
playstation console 1
playstation anyone 1
playstation continue 1
playstation leaning 1
playstation create 1
playstation v 3
playstation living 1
playstation grown 1
playstation sooo 1
playstation controller 1
playstation chap 1


In [99]:
from tkinter import *

def update_suggestions(event):
  prefix = entry.get()
  suggestions.set(suggest_word(prefix))

window = Tk()
window.title("Search Bar")

entry = Entry(window, width=50)
entry.bind("<KeyRelease>", update_suggestions)
entry.pack()

suggestions = StringVar()
suggestion_label = Label(window, textvariable=suggestions)
suggestion_label.pack()

window.mainloop()


p solely 1
p plus 6
p perfect 1
p couldnt 1
p let 1
p xsx 2
p man 2
p baby 1
p respect 1
p fan 4
p want 1
p however 2
p p 26
p current 1
p new 3
p playing 1
p ive 5
p ingame 1
p wish 2
p xbox 37
p constantly 1
p lot 1
p v 40
p still 7
p slim 5
p gow 1
p console 7
p game 9
p content 1
p seems 2
p make 1
p like 6
p friend 4
p exclusive 15
p regardless 1
p rpg 1
p look 5
p get 2
p played 2
p say 2
p software 1
p first 4
p used 1
p fps 2
p dont 8
p loved 2
p somewhere 1
p couple 1
p guy 3
p pro 7
p bought 2
p really 3
p anyway 1
p doesnt 5
p time 1
p come 2
p sooner 1
p rift 1
p vid 1
p cool 4
p familiar 1
p better 16
p switched 3
p ever 3
p grew 1
p definitely 1
p huge 2
p blend 1
p last 3
p play 2
p stand 1
p came 2
p got 6
p love 4
p im 8
p masterpiece 1
p sugestion 1
p great 5
p decade 1
p amazing 1
p video 1
p ability 1
p ill 4
p day 6
p using 1
p controller 19
p original 1
p lol 1
p thank 1
p brings 1
p sony 1
p long 1
p mate 1
p moved 1
p series 4
p hz 1
p data 1
p gon 1
p youre 1
p